<a href="https://colab.research.google.com/github/juliakarabasova/labs_and_projects/blob/main/nlp/text_representations_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymystem3==0.1.10

  Attempting uninstall: pymystem3
    Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0


In [3]:
import os
import re
import math
import pandas as pd
import nltk
from string import punctuation
from pymystem3 import Mystem
import numpy as np

### 1. Get links to all files in dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
os.listdir('/content/drive/MyDrive/dataset')

['.DS_Store', 'bad', 'good', 'neutral']

In [ ]:
list_of_files = []
for dirname in ['bad', 'good', 'neutral']:
    for filename in os.listdir(r"/content/drive/MyDrive/dataset" + r'/' + dirname):
        list_of_files.append(dirname + "/" + filename)

In [ ]:
list_of_files[:5]

['bad/785.txt', 'bad/253.txt', 'bad/330.txt', 'bad/592.txt', 'bad/83.txt']

In [ ]:
len(list_of_files)

2900

### 2. Form a dataset

In [ ]:
df = pd.DataFrame(columns=["text", "target"])

for filename in list_of_files:
    text = open(os.path.join("/content/drive/MyDrive/dataset", filename), encoding='utf8', errors='ignore').read()
    label = filename.split('/')[0]
    df = df.append({
                    "text": text,
                    "target":  label
                    }, ignore_index=True)
    
df

,text,target
0,"Пародия пародии рознь. Эту пародию мне бы, на ...",bad
1,В один прекрасный день я решил посмотреть паро...,bad
2,В поисках хорошего фильма на вечер на какие то...,bad
3,"По сравнению с этим фильмом, даже 'Борат' вам ...",bad
4,"Бррр... Ну нет, я такие фильмы терпеть не наме...",bad
...,...,...
2895,Помню в детстве у меня был диск с мультфильмам...,neutral
2896,Да-а...Посмотрел очередную поделку от мега гиг...,neutral
2897,"Волшебно, ярко и зрелищно.\n\n\nНостальгия сде...",neutral
2898,"Сюжет, я думаю, рассказывать никому не надо. П...",neutral


In [ ]:
df_copy = df.copy()

### 3. Preprocess texts

In [ ]:
m = Mystem()

def preprocess(text):
    lemmas = m.lemmatize(text)
    return [l.strip() for l in lemmas if l.strip() != '']

print(preprocess(df['text'][0]))

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


['пародия', 'пародия', 'рознь', '.', 'этот', 'пародия', 'я', 'бы', ',', 'на', 'место', 'режиссер', ',', 'стыдно', 'быть', 'бы', 'выставлять', 'на', 'суд', 'публика', '.', 'нелепо', 'до', 'безумие', ',', 'актер', 'играть', 'так', ',', 'что', 'плакать', 'хотеться', ',', 'а', 'добротный', 'юмор', 'здесь', 'и', 'вовсе', 'не', 'пахнуть', '.', 'не', 'вполне', 'понятный', 'событие', 'быстро', 'сменять', 'друг', 'друг', ',', 'даже', 'не', 'оставлять', 'после', 'себя', 'абсолютно', 'никакой', 'осадок', 'в', 'память', 'у', 'зритель', '.', 'любитель', 'от', 'душа', 'посмеяться', 'в', 'этот', 'фильм', 'не', 'находить', 'искомый', '.', 'задумка', ',', 'может', ',', 'быть', 'и', 'неплохой', '-', 'кроссовер', ',', 'эдакий', 'пересечение', 'разный', 'сюжет', ',', 'абсолютно', 'несовместимый', 'между', 'себя', 'фильм', '.', 'но', 'то', ',', 'как', 'этот', 'задумка', 'реализовать', ',', 'действительно', 'шокировать', '.', '1', 'из', '10', 'просто', 'потратить', 'впустую', 'полтора', 'час', 'жизнь', '.',

In [ ]:
df["text"] = df["text"].apply(preprocess)
df

,text,target
0,"[пародия, пародия, рознь, ., этот, пародия, я,...",bad
1,"[в, один, прекрасный, день, я, решать, посмотр...",bad
2,"[в, поиск, хороший, фильм, на, вечер, на, како...",bad
3,"[по, сравнение, с, этот, фильм, ,, даже, ', бо...",bad
4,"[Бррр, ..., ну, нет, ,, я, такой, фильм, терпе...",bad
...,...,...
2895,"[помнить, в, детство, у, я, быть, диск, с, мул...",neutral
2896,"[да, -, а, ..., посмотреть, очередной, поделка...",neutral
2897,"[волшебно, ,, ярко, и, зрелищно, ., ностальгия...",neutral
2898,"[сюжет, ,, я, думать, ,, рассказывать, никто, ...",neutral


### 4. Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vectorizer = CountVectorizer()

corpus = [
        'This is the first document.',
        'This is the second second document.',
        'And the third one.',
        'Is this the first document?',
    ]
X = count_vectorizer.fit_transform(corpus)
X

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [ ]:
count_vectorizer.get_feature_names_out()

array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], dtype=object)

In [ ]:
X.toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]])

In [ ]:
df_joined = df.copy()
df_joined["text"] = df_joined["text"].apply(lambda x: ' '.join(x))
df_joined

,text,target
0,"пародия пародия рознь . этот пародия я бы , на...",bad
1,в один прекрасный день я решать посмотреть пар...,bad
2,в поиск хороший фильм на вечер на какой только...,bad
3,"по сравнение с этот фильм , даже ' борат ' вы ...",bad
4,"Бррр ... ну нет , я такой фильм терпеть не нам...",bad
...,...,...
2895,помнить в детство у я быть диск с мультфильм д...,neutral
2896,да - а ... посмотреть очередной поделка от мег...,neutral
2897,"волшебно , ярко и зрелищно . ностальгия сделат...",neutral
2898,"сюжет , я думать , рассказывать никто не надо ...",neutral


In [ ]:
count_vectorizer = CountVectorizer()

X_bow = count_vectorizer.fit_transform(df_joined["text"])
X_bow

<2900x32557 sparse matrix of type '<class 'numpy.int64'>'
	with 646987 stored elements in Compressed Sparse Row format>

In [ ]:
count_vectorizer.get_feature_names_out()

array(['00', '000', '00000001', ..., 'ёх', 'ёёёё', 'єти'], dtype=object)

In [ ]:
X_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 5. tf*idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
corpus = df_joined.copy()
corpus

,text,target
0,"пародия пародия рознь . этот пародия я бы , на...",bad
1,в один прекрасный день я решать посмотреть пар...,bad
2,в поиск хороший фильм на вечер на какой только...,bad
3,"по сравнение с этот фильм , даже ' борат ' вы ...",bad
4,"Бррр ... ну нет , я такой фильм терпеть не нам...",bad
...,...,...
2895,помнить в детство у я быть диск с мультфильм д...,neutral
2896,да - а ... посмотреть очередной поделка от мег...,neutral
2897,"волшебно , ярко и зрелищно . ностальгия сделат...",neutral
2898,"сюжет , я думать , рассказывать никто не надо ...",neutral


In [ ]:
tfidf_vectorizer = TfidfVectorizer()

X = tfidf_vectorizer.fit_transform(corpus['text'])
X

<2900x32534 sparse matrix of type '<class 'numpy.float64'>'
	with 646834 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_vectorizer.get_feature_names_out()

array(['00', '000', '00000001', ..., 'ёх', 'ёёёё', 'єти'], dtype=object)

In [ ]:
X.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
tfidf_vectorizer = CountVectorizer()

X_tfidf = tfidf_vectorizer.fit_transform(df_joined["text"])
X_tfidf.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 6. Word2Vec

#### Обучение модели

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
#window -- количество слов в контексте, на которые обращается внимание
#min_count -- модель игнорирует слова, встретившиеся в корпусе меньшее число раз 
#size -- размерность вектора
my_model = Word2Vec(list(df['text']), window=5, min_count=5, size=100)

In [ ]:
my_model.wv['фильм']

array([ 2.2802020e-03, -1.9405760e-01,  4.0112257e-01,  1.2461741e+00,
       -2.2641298e-01,  6.1628366e-01, -1.8630598e+00,  6.6823936e-01,
        8.5116845e-01, -1.1730500e+00, -7.3582149e-01, -1.0730895e+00,
        1.9445051e-02, -1.0103836e+00,  9.7599882e-01, -1.1297419e+00,
       -1.1491963e+00,  1.4155447e+00,  3.9041656e-01,  1.0592442e+00,
        6.8080472e-03, -6.3620859e-01,  3.4122363e-02,  1.5659435e+00,
       -7.5569344e-01, -4.7601095e-01, -7.1978742e-01,  1.3785312e+00,
        3.7313059e-01, -8.5273877e-02,  1.6153996e+00, -1.1481996e+00,
       -3.2374728e-01,  1.2820785e+00,  1.6255675e-01, -1.0363075e+00,
        4.3077806e-01,  4.0373477e-01, -8.2241940e-01,  3.3562279e-01,
        3.1102338e-01, -3.8331315e-01,  3.8312870e-01, -8.7173694e-01,
       -3.5879335e-01,  2.6399791e-01, -1.0770000e-01, -3.7695372e-01,
        2.1128374e-01,  1.6031953e+00, -3.7921891e-01,  8.5414517e-01,
       -7.3573691e-01, -3.5402700e-01, -1.1038983e+00,  2.2552280e+00,
      

In [ ]:
my_model.wv.most_similar(positive=['фильм'])

[('картина', 0.8715606331825256),
 ('сериал', 0.851007342338562),
 ('лента', 0.8140319585800171),
 ('кино', 0.7327029705047607),
 ('продолжение', 0.7003697156906128),
 ('блокбастер', 0.6667990684509277),
 ('проект', 0.6552362442016602),
 ('трейлер', 0.6537940502166748),
 ('шедевр', 0.6501321792602539),
 ('творение', 0.6392271518707275)]

In [ ]:
my_model.wv.most_similar(positive=['хороший'])

[('плохой', 0.7911492586135864),
 ('достойный', 0.7214312553405762),
 ('безусловно', 0.6682688593864441),
 ('пожалуй', 0.6643847823143005),
 ('талантливый', 0.6626682281494141),
 ('удачный', 0.6546406149864197),
 ('замечательный', 0.6475631594657898),
 ('отличный', 0.6416290998458862),
 ('послужной', 0.6049915552139282),
 ('идеальный', 0.6046740412712097)]

In [ ]:
my_model.wv.most_similar(positive=['интересный', 'захватывающий'], negative=['скучный'])

In [ ]:
my_model.wv.similarity('сюжет', 'интересный')

0.698859

In [ ]:
my_model.wv.similarity('сюжет', 'красивый')

0.3383449

In [ ]:
my_model.wv.doesnt_match(['фильм', 'кино', 'сериал', 'актер'])

'актер'

In [ ]:
my_model.wv.doesnt_match(['боевик', 'триллер', 'комедия', 'детектив'])

'детектив'

Можно сохранить модель целиком (чтобы дообучить ее позже)

In [ ]:
my_model.save("my_model.model")
# Word2Vec.load("word2vec.model")

Можно сохранить только вектора

In [ ]:
word_vectors = my_model.wv
word_vectors.save("my_model.wordvectors")

# wv = KeyedVectors.load("my_model.wordvectors")

#### Загрузка предобученной модели

https://rusvectores.org/ru/models/

In [ ]:
model_rus = KeyedVectors.load_word2vec_format('ruscorpora_upos_skipgram_300_5_2018.vec', binary=False)

In [ ]:
model_rus['кошка_NOUN']

array([ 5.38850e-02,  7.68320e-02, -2.08360e-02,  5.04660e-02,
       -1.83440e-02,  2.49620e-02,  2.22730e-02, -6.97660e-02,
       -6.77290e-02,  2.67250e-02, -1.93420e-02, -7.69620e-02,
       -1.65210e-02,  3.68000e-02,  7.34000e-02,  1.81910e-02,
       -2.90160e-02,  7.20770e-02,  5.62100e-03,  1.33670e-02,
        3.72470e-02,  2.65340e-02, -2.65480e-02,  1.36221e-01,
        2.91200e-03, -2.46040e-02, -2.38830e-02, -3.06300e-02,
       -8.69400e-03, -1.08803e-01,  1.01105e-01, -3.06730e-02,
       -1.68290e-02,  8.18290e-02, -1.94850e-02,  5.38960e-02,
       -8.13300e-03,  4.86310e-02, -2.69210e-02,  7.38900e-03,
        1.50000e-05, -4.95700e-02,  1.27940e-02,  5.31210e-02,
       -2.08490e-02, -1.06714e-01,  1.04160e-02,  7.26030e-02,
       -4.03220e-02,  4.04200e-02, -3.42690e-02, -5.27990e-02,
        5.67580e-02,  1.74870e-02, -3.68720e-02, -3.24450e-02,
       -4.96150e-02, -6.17700e-03,  8.90000e-04, -4.65600e-03,
        5.63450e-02,  6.86770e-02,  6.10540e-02, -1.063

In [ ]:
model_rus.most_similar(positive=['кошка_NOUN'])

[('кот_NOUN', 0.780572235584259),
 ('собака_NOUN', 0.7084057331085205),
 ('котенок_NOUN', 0.6833623647689819),
 ('мяукать_VERB', 0.6613745093345642),
 ('мяукать_NOUN', 0.6479178071022034),
 ('клеточка_PROPN', 0.642410397529602),
 ('тобик_PROPN', 0.6401646137237549),
 ('крыса_NOUN', 0.635063648223877),
 ('котенка_NOUN', 0.629923939704895),
 ('окотиться_VERB', 0.6232213377952576)]

Добавление частеречного тега

In [ ]:
import ufal.udpipe as ud

https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-2859

In [ ]:
model_rus_tag = ud.Model.load('ud-2.2-conll18-baseline-models/models/russian-syntagrus-ud-2.2-conll18-180430.udpipe')
pipeline = ud.Pipeline(model_rus_tag, "horizontal", ud.Pipeline.DEFAULT, ud.Pipeline.DEFAULT, "conllu")

In [ ]:
print(pipeline.process('мама с папой мыли раму и ковер, а я ходил гулять'))

# newdoc
# newpar
# sent_id = 1
1	мама	мама	NOUN	_	Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing	4	nsubj	_	_
2	с	с	ADP	_	_	3	case	_	_
3	папой	папа	NOUN	_	Animacy=Anim|Case=Ins|Gender=Masc|Number=Sing	1	nmod	_	_
4	мыли	мыть	VERB	_	Aspect=Imp|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
5	раму	рама	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	4	obj	_	_
6	и	и	CCONJ	_	_	10	cc	_	_
7	ковер,	ковер,	ADV	_	Degree=Pos	4	advmod	_	_
8	а	а	CCONJ	_	_	10	cc	_	_
9	я	я	PRON	_	Case=Nom|Number=Sing|Person=1	10	nsubj	_	_
10	ходил	ходить	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	7	conj	_	_
11	гулять	гулять	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Act	10	xcomp	_	_




In [ ]:
def add_utag(word, pipeline):
    #Add POS tag to word using UDPipe
    return '_'.join(re.findall('\n1\\t(\w+)\\t\w+\\t([A-Z]+)', pipeline.process(word))[0])

In [ ]:
model_rus.most_similar(positive=[add_utag('кошка', pipeline)])

[('кот_NOUN', 0.7805721759796143),
 ('собака_NOUN', 0.7084056735038757),
 ('котенок_NOUN', 0.6833623647689819),
 ('мяукать_VERB', 0.661374568939209),
 ('мяукать_NOUN', 0.6479178071022034),
 ('клеточка_PROPN', 0.642410397529602),
 ('тобик_PROPN', 0.6401645541191101),
 ('крыса_NOUN', 0.635063648223877),
 ('котенка_NOUN', 0.629923939704895),
 ('окотиться_VERB', 0.6232213973999023)]

https://nlp.stanford.edu/projects/glove/

In [ ]:
w2v_english = KeyedVectors.load_word2vec_format("glove.twitter.27B.25d_mod.txt", binary=False)

In [ ]:
w2v_english['computer']

array([ 0.64005 , -0.019514,  0.70148 , -0.66123 ,  1.1723  , -0.58859 ,
        0.25917 , -0.81541 ,  1.1708  ,  1.1413  , -0.15405 , -0.11369 ,
       -3.8414  , -0.87233 ,  0.47489 ,  1.1541  ,  0.97678 ,  1.1107  ,
       -0.14572 , -0.52013 , -0.52234 , -0.92349 ,  0.34651 ,  0.061939,
       -0.57375 ], dtype=float32)

In [ ]:
w2v_english.most_similar(positive=['apple'])

[('windows', 0.8948712944984436),
 ('microsoft', 0.8858076333999634),
 ('google', 0.8823867440223694),
 ('galaxy', 0.8806391358375549),
 ('flash', 0.8793812394142151),
 ('android', 0.8782057762145996),
 ('nokia', 0.8770236372947693),
 ('samsung', 0.8697316646575928),
 ('chrome', 0.8691699504852295),
 ('ipad', 0.8670315742492676)]

In [ ]:
w2v_english.similarity('iphone', 'apple')

0.846078

In [ ]:
w2v_english.similarity('iphone', 'microsoft')

0.71931857

In [ ]:
w2v_english.similarity('iphone', 'nivea')

0.17504907

### 8. Fasttext

In [ ]:
from gensim.models import FastText

fasttext_model = FastText(vector_size=100, window=3, min_count=1, sentences=list(df["text"]), epochs=10)

In [ ]:
fasttext_model.wv["фильм"]

array([ 0.5362766 ,  0.45565292,  1.9937762 , -0.32736206,  1.6903409 ,
        0.44180977, -0.41248763, -0.03373772,  0.37971368, -0.20872155,
        0.247941  ,  2.7339318 ,  1.9529096 ,  0.16145045,  0.9461472 ,
        1.2783732 , -0.21335469, -0.96383613, -1.1248935 ,  0.07255437,
        0.2806648 ,  0.37223923,  0.5949794 , -0.76535714, -0.36244804,
       -0.31470576,  1.7604123 ,  1.6395363 , -0.6039507 ,  0.08312204,
       -2.3963752 , -2.1767733 ,  0.07692423, -0.28875962,  1.865387  ,
       -0.14657733, -0.66610354,  0.2300724 , -1.6550189 ,  0.86700886,
       -0.82161254,  2.4829023 , -1.1599786 ,  3.0888877 , -0.21374995,
        1.0124897 , -1.8474824 , -0.5371141 , -0.72643864, -0.5265844 ,
       -0.24170473, -2.344123  ,  1.1717926 ,  1.8583089 , -1.4834018 ,
       -1.1791576 , -4.115584  , -0.6078417 , -1.2843875 ,  0.69987565,
       -1.9712173 , -0.51217616, -0.6512755 , -1.6742693 ,  1.792393  ,
       -0.4241422 , -0.7285652 ,  0.41388288, -1.5708936 , -0.66

Word vectors --> document vectors?

1. Elementwise sum/mean (https://stackoverflow.com/questions/44786174/how-combine-word-embedded-vectors-to-one-vector)<br>
2. Take the biggest number from each dimension <br>
3. ...

### 7. Doc2Vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(list(df['text']))]
d2v_model = Doc2Vec(documents, vector_size=100, window=2, min_count=1)

In [ ]:
d2v_model.infer_vector(df['text'][0])

array([ 0.16184753, -0.00140766, -0.34360182, -0.09366103,  0.05927211,
       -0.15132274,  0.12774721,  0.65085983, -0.47597265,  0.3290359 ,
       -0.10253838, -0.6176733 ,  0.18384029, -0.28245527, -0.37965995,
       -0.87344456, -0.3325106 , -0.27250615, -0.40843076, -0.07354342,
        0.3956681 ,  0.3804773 ,  0.23613517, -0.337306  ,  0.6176508 ,
        0.41763288, -0.8125513 ,  0.3767591 , -0.11818262, -0.72782   ,
        0.48191288,  0.80036175,  0.40251124,  0.01838744, -0.27582493,
        0.23302993, -0.27372605, -0.3889638 ,  0.08391088,  0.416919  ,
        0.47155398,  0.28860438,  0.05331756, -0.7967367 , -0.13589714,
       -0.06687639,  0.13683958,  0.19945462,  0.33725664,  0.39183092,
       -0.46517223, -0.4739014 ,  0.21961428,  0.22662234, -0.24649704,
        0.7926873 ,  0.31522694,  0.34957817,  0.03927494, -0.00405006,
        0.15417615,  0.60394704, -0.10514691,  0.44360417, -0.4884961 ,
        0.42794305,  0.21498704,  0.46358553, -0.7910037 ,  0.29

# Задание: <br>
1. Представить тексты в виде векторов и обучить на них классификатор LogisticRegression. <br>
2. Попробовать несколько представлений и понять, какое лучше работает. <br>
3. Предсказать классы для тестовой части датасета. <br>

## Preprocessing

In [ ]:
!pip install pymystem3==0.1.10

In [ ]:
!pip install --upgrade spacy

In [ ]:
!python3.7 -m spacy download en_core_web_sm
!python3.7 -m spacy download ru_core_news_sm

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from pymystem3 import Mystem

import nltk
import spacy

nltk.download('punkt')
nltk.download('stopwords')

class TextPreprocess:
  def __init__(self, text: str):
    self.text = text
  
  @staticmethod
  def tokenize(text):
    text = text.replace('\n', ' ')
    #tokens = text_ru.split()
    return word_tokenize(text)
  
  @staticmethod
  def remove_punctuation(text: list):
    return [w.strip(punctuation) for w in text]
  
  @staticmethod
  def to_lower_case(text: list):
    return [w.lower() for w in text if w != '']

  @staticmethod
  def remove_stop_words(text: list, lang: str):
    sw = stopwords.words(lang)
    return [w for w in text if w not in sw]
  
  @staticmethod
  def general_process(text: str):
    tokens = TextPreprocess.tokenize(text)
    no_punkt = TextPreprocess.remove_punctuation(tokens) 
    return TextPreprocess.to_lower_case(no_punkt)
  
  def preprocess_rus(self):
    clean_words = TextPreprocess.general_process(self.text)
    no_sw = TextPreprocess.remove_stop_words(clean_words, 'russian')

    nlp_rus = spacy.load("ru_core_news_sm")
    spacy_text = nlp_rus(' '.join(no_sw))
    lemmas = [token.lemma_ for token in spacy_text if token.pos_ != "SPACE"]

    return lemmas
  
  def preprocess_eng(self):
    clean_words = TextPreprocess.general_process(self.text)
    no_sw = TextPreprocess.remove_stop_words(clean_words, 'english')   

    nlp_eng = spacy.load("en_core_web_sm")
    spacy_text = nlp_eng(' '.join(no_sw))
    lemmas = [token.lemma_ for token in spacy_text if token.pos_ != "SPACE"]

    return lemmas 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def use_class(text):
  text_preproc = TextPreprocess(text)
  return text_preproc.preprocess_rus()

In [ ]:
df_copy["text"] = df_copy["text"].apply(use_class)  #check MyStem
df_copy

KeyboardInterrupt: ignored

## Start experiments

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
df_joined = df.copy()
df_joined["text"] = df_joined["text"].apply(lambda x: ' '.join(x))
df_joined

,text,target
0,"пародия пародия рознь . этот пародия я бы , на...",bad
1,в один прекрасный день я решать посмотреть пар...,bad
2,в поиск хороший фильм на вечер на какой только...,bad
3,"по сравнение с этот фильм , даже ' борат ' вы ...",bad
4,"Бррр ... ну нет , я такой фильм терпеть не нам...",bad
...,...,...
2895,помнить в детство у я быть диск с мультфильм д...,neutral
2896,да - а ... посмотреть очередной поделка от мег...,neutral
2897,"волшебно , ярко и зрелищно . ностальгия сделат...",neutral
2898,"сюжет , я думать , рассказывать никто не надо ...",neutral


In [ ]:
#X = np.array(list(df_bow["text"]))

d = {'bad': 0, 'good': 1, 'neutral': 2}
y = np.array([d[i] for i in df_joined["target"]])

test_indexes = np.random.randint(low=0, high=2899, size=(200))
train_indexes = np.array(list(set([i for i in range(2900)]) - set(test_indexes)))

### Initialize Vectorizer 1. Bag Of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vectorizer = CountVectorizer()

X_cv = count_vectorizer.fit_transform(df_joined['text'])
X_cv

<2900x32534 sparse matrix of type '<class 'numpy.int64'>'
	with 646834 stored elements in Compressed Sparse Row format>

In [ ]:
X_cv.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Train and test Vecotrizer 1.

In [ ]:
X_train = X_cv[train_indexes]
y_train = y[train_indexes]
X_test = X_cv[test_indexes]
y_test = y[test_indexes]

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=150).fit(X_train, y_train)
predictions = clf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
accuracy_score(y_test, predictions)

0.6

### Initialize Vectorizer 2. TdIdf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

X_tf = tfidf_vectorizer.fit_transform(df_joined['text'])
X_tf

<2900x32534 sparse matrix of type '<class 'numpy.float64'>'
	with 646834 stored elements in Compressed Sparse Row format>

In [ ]:
X_tf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Train and test Vectorizer 2.

In [ ]:
X_train = X_tf[train_indexes]
y_train = y[train_indexes]
X_test = X_tf[test_indexes]
y_test = y[test_indexes]

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=150).fit(X_train, y_train)
predictions = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, predictions)

0.64

### Initialize Vecotrizer 3. Doc2vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(list(df['text']))]

### Train and test Vectorizer 3.

In [ ]:
X_train = documents[train_indexes]
y_train = y[train_indexes]
X_test = documents[test_indexes]
y_test = y[test_indexes]

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=150).fit(X_train, y_train)
predictions = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, predictions)

0.64